# StackOverflow assistant bot

The goal of this notebook is to help us construct a *dialogue chat bot*, which will be able to:

* answer programming-related questions (using StackOverflow dataset);
* chit-chat and simulate dialogue on all non programming-related questions.

For a chit-chat mode we will use a pre-trained neural network engine available from [ChatterBot](https://github.com/gunthercox/ChatterBot).


In [1]:
! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py

--2019-12-21 14:01:22--  https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1991 (1.9K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   1.94K  --.-KB/s    in 0s      

2019-12-21 14:01:27 (42.7 MB/s) - ‘setup_google_colab.py’ saved [1991/1991]



In [0]:
import setup_google_colab

In [0]:
setup_google_colab.setup_project()

### Data description

To detect *intent* of users questions we will need two text collections:
- `tagged_posts.tsv` � StackOverflow posts, tagged with one programming language (*positive samples*).
- `dialogues.tsv` � dialogue phrases from movie subtitles (*negative samples*).


In [4]:
import sys
sys.path.append("..")
from common.download_utils import download_project_resources

download_project_resources()

For those questions, that have programming-related intent, we will proceed as follow predict programming language (only one tag per question allowed here) and rank candidates within the tag using embeddings.
For the ranking part, you will need:
- `word_embeddings.tsv` � word embeddings, that are trained with StarSpace.

As a result of this notebook, you should obtain the following new objects that you will then use in the running bot:

- `intent_recognizer.pkl` � intent recognition model;
- `tag_classifier.pkl` � programming language classification model;
- `tfidf_vectorizer.pkl` � vectorizer used during training;
- `thread_embeddings_by_tags` � folder with thread embeddings, arranged by tags.
    

Some functions will be reused by this notebook and the scripts, so we put them into *utils.py* file. Don't forget to open it and fill in the gaps!

In [5]:
!ls

boost_1_63_0.zip  dialogue_manager.py  sample_data	      utils.py
common		  main_bot.py	       setup_google_colab.py
data		  __pycache__	       Starspace


In [6]:
!ls data

dialogues.tsv  tagged_posts.tsv


In [7]:
! pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



### Downloading starspace embeddings generated earlier specifically from the dataset containing stackoverflow posts. Refer the notebook Starspace_embeddings_Stackoverflow.ipynb

In [0]:
downloaded = drive.CreateFile({'id': '1fl64k63LFUOuMZKxGzE1KVvMT5nwmlOU'})
downloaded.GetContentFile('my_starspace_embeddings_0612.tsv')

In [0]:
cp my_starspace_embeddings_0612.tsv data/word_embeddings.tsv

**We need to update utils.py. So here i am removing utils.py downloaded from project resources and uploading my utils.py "updated" with necessary functions!!!**

In [0]:
!rm utils.py

In [11]:
from google.colab import files
files.upload() 

Saving utils.py to utils.py


{'utils.py': b'\nimport nltk\nimport pickle\nimport re\nimport numpy as np\n\nnltk.download(\'stopwords\')\nfrom nltk.corpus import stopwords\n\n# Paths for all resources for the bot.\nRESOURCE_PATH = {\n    \'INTENT_RECOGNIZER\': \'intent_recognizer.pkl\',\n    \'TAG_CLASSIFIER\': \'tag_classifier.pkl\',\n    \'TFIDF_VECTORIZER\': \'tfidf_vectorizer.pkl\',\n    \'THREAD_EMBEDDINGS_FOLDER\': \'thread_embeddings_by_tags\',\n    \'WORD_EMBEDDINGS\': \'word_embeddings.tsv\',\n}\n\n\ndef text_prepare(text):\n    """Performs tokenization and simple preprocessing."""\n    \n    replace_by_space_re = re.compile(\'[/(){}\\[\\]\\|@,;]\')\n    bad_symbols_re = re.compile(\'[^0-9a-z #+_]\')\n    stopwords_set = set(stopwords.words(\'english\'))\n\n    text = text.lower()\n    text = replace_by_space_re.sub(\' \', text)\n    text = bad_symbols_re.sub(\'\', text)\n    text = \' \'.join([x for x in text.split() if x and x not in stopwords_set])\n\n    return text.strip()\n\n\ndef load_embeddings(emb

In [12]:
from utils import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
!ls

adc.json	  dialogue_manager.py		    sample_data
boost_1_63_0.zip  main_bot.py			    setup_google_colab.py
common		  my_starspace_embeddings_0612.tsv  Starspace
data		  __pycache__			    utils.py


In [14]:
!ls data

dialogues.tsv  tagged_posts.tsv  word_embeddings.tsv


## Part I. Intent and language recognition

We want to write a bot, which will not only **answer programming-related questions**, but also will be able to **maintain a dialogue**. We would also like to detect the *intent* of the user from the question (we could have had a 'Question answering mode' check-box in the bot, but it wouldn't fun at all, would it?). So the first thing we need to do is to **distinguish programming-related questions from general ones**.

It would also be good to predict which programming language a particular question referees to. By doing so, we will speed up question search by a factor of the number of languages (10 here), and exercise our *text classification* skill a bit. :)

In [0]:
import numpy as np
import pandas as pd
import pickle
import re

from sklearn.feature_extraction.text import TfidfVectorizer

### Data preparation

In the first assignment (Predict tags on StackOverflow with linear models), you have already learnt how to preprocess texts and do TF-IDF tranformations. Reuse your code here. In addition, you will also need to [dump](https://docs.python.org/3/library/pickle.html#pickle.dump) the TF-IDF vectorizer with pickle to use it later in the running bot.

In [0]:
def tfidf_features(X_train, X_test, vectorizer_path):
    """Performs TF-IDF transformation and dumps the model."""
    
    # Train a vectorizer on X_train data.
    # Transform X_train and X_test data.
    
    # Pickle the trained vectorizer to 'vectorizer_path'
    # Don't forget to open the file in writing bytes mode.
    
    ######################################
    ######### YOUR CODE HERE #############
    tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, token_pattern = '(\S+)', ngram_range=(1, 2))
    X_train = tfidf_vectorizer.fit_transform(X_train)
    ######################################
    
    X_test = tfidf_vectorizer.transform(X_test)
    ######################################
    
    file_Name = vectorizer_path
    # open the file for writing
    fileObject = open(file_Name,'wb') 

    # this writes the object a to the
    # file named 'testfile'
    pickle.dump(tfidf_vectorizer,fileObject)   

    # here we close the fileObject
    fileObject.close()
    
    ######################################
    
    return X_train, X_test

Now, load examples of two classes. Use a subsample of stackoverflow data to balance the classes. You will need the full data later.

In [0]:
sample_size = 200000

dialogue_df = pd.read_csv('data/dialogues.tsv', sep='\t').sample(sample_size, random_state=0)
stackoverflow_df = pd.read_csv('data/tagged_posts.tsv', sep='\t').sample(sample_size, random_state=0)

Check how the data look like:

In [18]:
dialogue_df.head()

,text,tag
82925,"Donna, you are a muffin.",dialogue
48774,He was here last night till about two o'clock....,dialogue
55394,"All right, then make an appointment with her s...",dialogue
90806,"Hey, what is this-an interview? We're supposed...",dialogue
107758,Yeah. He's just a friend of mine I was trying ...,dialogue


In [19]:
stackoverflow_df.head()

,post_id,title,tag
2168983,43837842,Efficient Algorithm to compose valid expressio...,python
1084095,15747223,Why does this basic thread program fail with C...,c_cpp
1049020,15189594,Link to scroll to top not working,javascript
200466,3273927,Is it possible to implement ping on windows ph...,c#
1200249,17684551,GLSL normal mapping issue,c_cpp


Apply *text_prepare* function to preprocess the data:

In [0]:
from utils import text_prepare

In [0]:
dialogue_df['text'] = [text_prepare(x) for x in list(dialogue_df['text'])]
stackoverflow_df['title'] = [text_prepare(x) for x in list(stackoverflow_df['title'])]

In [22]:
dialogue_df.head()

,text,tag
82925,donna muffin,dialogue
48774,last night till two oclock hear really got stu...,dialogue
55394,right make appointment see,dialogue
90806,hey thisan interview supposed making love,dialogue
107758,yeah hes friend mine trying help,dialogue


In [23]:
stackoverflow_df.head()

,post_id,title,tag
2168983,43837842,efficient algorithm compose valid expressions ...,python
1084095,15747223,basic thread program fail clang pass g++,c_cpp
1049020,15189594,link scroll top working,javascript
200466,3273927,possible implement ping windows phone 7,c#
1200249,17684551,glsl normal mapping issue,c_cpp


### Intent recognition

We will do a binary classification on TF-IDF representations of texts. Labels will be either `dialogue` for general questions or `stackoverflow` for programming-related questions. First, prepare the data for this task:
- concatenate `dialogue` and `stackoverflow` examples into one sample
- split it into train and test in proportion 9:1, use *random_state=0* for reproducibility
- transform it into TF-IDF features

In [0]:
from sklearn.model_selection import train_test_split

In [25]:
X = np.concatenate([dialogue_df['text'].values, stackoverflow_df['title'].values])
y = ['dialogue'] * dialogue_df.shape[0] + ['stackoverflow'] * stackoverflow_df.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

X_train_tfidf, X_test_tfidf = tfidf_features(X_train, X_test, RESOURCE_PATH['TFIDF_VECTORIZER'])

Train size = 360000, test size = 40000


Train the **intent recognizer** using LogisticRegression on the train set with the following parameters: *penalty='l2'*, *C=10*, *random_state=0*. Print out the accuracy on the test set to check whether everything looks good.

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [27]:
######################################
intent_recognizer = LogisticRegression(penalty='l2', C=10, random_state=0)
intent_recognizer.fit(X_train_tfidf, y_train)
######################################

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
# Check test accuracy.
y_test_pred = intent_recognizer.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.991575


Dump the classifier to use it in the running bot.

In [0]:
pickle.dump(intent_recognizer, open(RESOURCE_PATH['INTENT_RECOGNIZER'], 'wb'))

### Programming language classification 

We will train one more classifier for the programming-related questions. It will predict exactly one tag (=programming language) and will be also based on Logistic Regression with TF-IDF features. 

First, let us prepare the data for this task.

In [0]:
X = stackoverflow_df['title'].values
y = stackoverflow_df['tag'].values

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

Train size = 160000, test size = 40000


Let us reuse the TF-IDF vectorizer that we have already created above. It should not make a huge difference which data was used to train it.

In [0]:
vectorizer = pickle.load(open(RESOURCE_PATH['TFIDF_VECTORIZER'], 'rb'))

X_train_tfidf, X_test_tfidf = vectorizer.transform(X_train), vectorizer.transform(X_test)

Train the **tag classifier** using OneVsRestClassifier wrapper over LogisticRegression. Use the following parameters: *penalty='l2'*, *C=5*, *random_state=0*.

In [0]:
from sklearn.multiclass import OneVsRestClassifier

In [34]:
######################################
tag_classifier = OneVsRestClassifier(LogisticRegression(penalty='l2', C=5, random_state=0))
tag_classifier.fit(X_train_tfidf, y_train)
######################################

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


OneVsRestClassifier(estimator=LogisticRegression(C=5, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=0, solver='warn',
                                                 tol=0.0001, verbose=0,
                                                 warm_start=False),
                    n_jobs=None)

In [35]:
# Check test accuracy.
y_test_pred = tag_classifier.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.800725


Dump the classifier to use it in the running bot.

In [0]:
pickle.dump(tag_classifier, open(RESOURCE_PATH['TAG_CLASSIFIER'], 'wb'))

## Part II. Ranking  questions with embeddings

To find a relevant answer (a thread from StackOverflow) on a question you will use vector representations to calculate similarity between the question and existing threads. We will need to use `question_to_vec` function prepared and made availabe in *utils.py* , which can create such a representation based on word vectors. 

However, it would be costly to compute such a representation for all possible answers in *online mode* of the bot (e.g. when bot is running and answering questions from many users). This is the reason why you will create a *database* with pre-computed representations. These representations will be arranged by non-overlaping tags (programming languages), so that the search of the answer can be performed only within one tag each time. This will make our bot even more efficient and allow not to store all the database in RAM. 

Load StarSpace embeddings which were trained on Stack Overflow posts. These embeddings were trained in *supervised mode* for duplicates detection on the same corpus that is used in search. We can account on that these representations will allow us to find closely related answers for a question. 



In [0]:
starspace_embeddings, embeddings_dim = load_embeddings('data/word_embeddings.tsv')

Since we want to precompute representations for all possible answers, we need to load the whole posts dataset, unlike we did for the intent classifier:

In [0]:
posts_df = pd.read_csv('data/tagged_posts.tsv', sep='\t')

Look at the distribution of posts for programming languages (tags) and find the most common ones. 
You might want to use pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) and [count](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.count.html) methods:

In [39]:
posts_df.head(5)

,post_id,title,tag
0,9,Calculate age in C#,c#
1,16,Filling a DataSet or DataTable from a LINQ que...,c#
2,39,Reliable timer in a console application,c#
3,42,Best way to allow plugins for a PHP application,php
4,59,"How do I get a distinct, ordered list of names...",c#


In [0]:
counts_by_tag = posts_df.groupby('tag').count()['title']

In [41]:
counts_by_tag

tag
c#            394451
c_cpp         281300
java          383456
javascript    375867
php           321752
python        208607
r              36359
ruby           99930
swift          34809
vb             35044
Name: title, dtype: int64

Now for each `tag` you need to create two data structures, which will serve as online search index:
* `tag_post_ids` � a list of post_ids with shape `(counts_by_tag[tag],)`. It will be needed to show the title and link to the thread;
* `tag_vectors` � a matrix with shape `(counts_by_tag[tag], embeddings_dim)` where embeddings for each answer are stored.

Implement the code which will calculate the mentioned structures and dump it to files. It should take several minutes to compute it.

In [0]:
import os
os.makedirs(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], exist_ok=True)

for tag, count in counts_by_tag.items():
    tag_posts = posts_df[posts_df['tag'] == tag]
    
    tag_post_ids = tag_posts['post_id'].values
    
    tag_vectors = np.zeros((count, embeddings_dim), dtype=np.float32)
    for i, title in enumerate(tag_posts['title']):
        tag_vectors[i, :] = question_to_vec(title, starspace_embeddings, embeddings_dim)

    # Dump post ids and vectors to a file.
    filename = os.path.join(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], os.path.normpath('%s.pkl' % tag))
    pickle.dump((tag_post_ids, tag_vectors), open(filename, 'wb'))

In [43]:
!ls

adc.json			  __pycache__
boost_1_63_0.zip		  sample_data
common				  setup_google_colab.py
data				  Starspace
dialogue_manager.py		  tag_classifier.pkl
intent_recognizer.pkl		  tfidf_vectorizer.pkl
main_bot.py			  thread_embeddings_by_tags
my_starspace_embeddings_0612.tsv  utils.py


In [44]:
!ls thread_embeddings_by_tags/

c_cpp.pkl  java.pkl	   php.pkl     r.pkl	 swift.pkl
c#.pkl	   javascript.pkl  python.pkl  ruby.pkl  vb.pkl


In [45]:
!zip -r /content/file.zip /content/thread_embeddings_by_tags

  adding: content/thread_embeddings_by_tags/ (stored 0%)
  adding: content/thread_embeddings_by_tags/ruby.pkl (deflated 16%)
  adding: content/thread_embeddings_by_tags/javascript.pkl (deflated 15%)
  adding: content/thread_embeddings_by_tags/php.pkl (deflated 15%)
  adding: content/thread_embeddings_by_tags/c#.pkl (deflated 18%)
  adding: content/thread_embeddings_by_tags/java.pkl (deflated 17%)
  adding: content/thread_embeddings_by_tags/vb.pkl (deflated 20%)
  adding: content/thread_embeddings_by_tags/c_cpp.pkl (deflated 16%)
  adding: content/thread_embeddings_by_tags/r.pkl (deflated 12%)
  adding: content/thread_embeddings_by_tags/swift.pkl (deflated 18%)
  adding: content/thread_embeddings_by_tags/python.pkl (deflated 15%)


In [46]:
from google.colab import files
files.download("/content/file.zip")

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 48244, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil